# Grocery Basket Items Counting Using OpenCV Python

In [1]:
import cv2

In [2]:
print(cv2.__version__)

3.4.0


In [3]:
import argparse
import datetime
import imutils
import time
import numpy as np
#from skvideo.io import VideoWriter

## Read Video File

In [4]:
cap = cv2.VideoCapture("C:\\Users\\212458792\\Music\\Downloads\\BasketVideos\\v1.mp4") #Open video file

count=0
w = cap.get(3) #get width
h = cap.get(4) #get height

mx = int(w/2)#to calculate the middle coordinates in our video (width/2, height/2).
my = int(h/2)


print(w)
print(h)
print(mx)
print(my)


1376.0
712.0
688
356


## Display Frames and their number

In [6]:
while True:
    ret, frame = cap.read()
    #print(frame.shape)
    count=count+1
    text="Frame count: "+str(count)
    
    if ret == True:
        cv2.putText(frame,text,(mx,my),cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,255),1,cv2.LINE_AA)
        cv2.imshow('frame',frame)

        if cv2.waitKey(5) & 0xFF == ord('q'):
            break

    else:
        break

cap.release()
cv2.destroyAllWindows()

# video 1 has a total of 740 frames

## Draw two horizontal lines as reference lines to count number of objects taken out and put into basket

In [5]:
while True:
    
    ret,frame=cap.read() #read a frame

    line1 = np.array([[0,100],[10000,100]], np.int32).reshape((-1,1,2))
    line2 = np.array([[0,600],[10000,600]], np.int32).reshape((-1,1,2))

    frame2 = cv2.polylines(frame,[line1],False,(255,0,0),thickness=2)
    frame2 = cv2.polylines(frame,[line2],False,(0,0,255),thickness=2)
    
    if ret == True:
        
        cv2.imshow('Frame 2',frame2)
       
        if cv2.waitKey(5) & 0xFF == ord('q'):
            break

    else:
        break

cap.release()
cv2.destroyAllWindows()

## Background Subtraction

In [5]:
#A background is considered to be as anything constant in a series of images, anything that stays static. 
#The foreground is everything that changes (moves).
 
fgbg = cv2.createBackgroundSubtractorMOG2(detectShadows = True) #Create the background substractor

size = (int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)),int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)))

fourcc = cv2.VideoWriter_fourcc(*"DIVX")

bgsubvideo = cv2.VideoWriter('C:\\Users\\212458792\\Music\\Downloads\\BasketVideos\\bgsubvideo.avi', fourcc, 10.0,size)

while True:
    
    ret, frame = cap.read()#read a frame
    
    if ret==True:
        
        fgmask = fgbg.apply(frame) #Use the substractor
        bgsubvideo.write(fgmask)
        #cv2.imshow('Background Substraction',fgmask) #show background subtracted video

    else:
        break

cap.release() #release video file
cv2.destroyAllWindows() #close all openCV windows
bgsubvideo.release()


# Morphological Transformations

##### Morphological transformations are some simple operations based on the image shape. It is normally performed on binary images. It needs two inputs, one is our original image, second one is called structuring element or kernel which decides the nature of operation.
##### Common Morphology operators: Erode and Dilate. What is Erosion and Dialation of an image?
##### Why do you need them? 
##### Read https://docs.opencv.org/3.0-beta/doc/py_tutorials/py_imgproc/py_morphological_ops/py_morphological_ops.html
##### Erosion followed by Dialation = Opening
##### Dialation followed by Opening = Closing

In [5]:
cap = cv2.VideoCapture("C:\\Users\\212458792\\Music\\Downloads\\BasketVideos\\bgsubvideo.avi") #Open video file

fourcc = cv2.VideoWriter_fourcc(*'DIVX')

size = (int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)),int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)))


out = cv2.VideoWriter('C:\\Users\\212458792\\Music\\Downloads\\BasketVideos\\morphed_out.avi',fourcc, 20.0, size,False)

#out = cv2.VideoWriter(str("C:\\Users\\212458792\\Music\\Downloads\\BasketVideos\\morphed_out.mp4"),fourcc(*'DIVX'),12, (1376,712),False)
 
# print(frame.shape)

while True:
    
    ret,frame=cap.read() #read a frame
    
    ret,thresh1 = cv2.threshold(frame,200,255,cv2.THRESH_BINARY)

    #kernel = np.ones((5,5),np.uint8)

    #opening = cv2.morphologyEx(thresh1, cv2.MORPH_OPEN, kernel)
    #closing = cv2.morphologyEx(opening, cv2.MORPH_CLOSE, kernel)
    
    
    if ret == True:
        
        #out.write(closing) # Write the frame into the file 'output.avi'
 
    # Display the resulting frame    
        cv2.imshow('frame',frame)
 
    # Press Q on keyboard to stop recording
    if cv2.waitKey(1) & 0xFF == ord('q'):
      break
 
  # Break the loop
    else:
      break 
 
# When everything done, release the video capture and video write objects
cap.release()
out.release()
 
# Closes all the frames
cv2.destroyAllWindows() 

## Draw Contours

In [5]:
fgbg = cv2.createBackgroundSubtractorMOG2(detectShadows = True) #Create the background substractor

kernelOp = np.ones((3,3),np.uint8)
kernelCl = np.ones((11,11),np.uint8)

while(cap.isOpened()):
    ret, frame = cap.read() #read a frame
    
    fgmask = fgbg.apply(frame) #Use the substractor
    
    ret,imBin= cv2.threshold(fgmask,200,255,cv2.THRESH_BINARY)
    #Opening (erode->dilate) para quitar ruido.
    mask = cv2.morphologyEx(imBin, cv2.MORPH_OPEN, kernelOp)
    #Closing (dilate -> erode) para juntar regiones blancas.
    mask = cv2.morphologyEx(mask , cv2.MORPH_CLOSE, kernelCl)
    
    _, contours0, hierarchy = cv2.findContours(mask,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_NONE) 
    #cv2.RETR_EXTERNAL means we only care about external contours (contours within contours will not be detected)
    #cv2.CHAIN_APPROX_NONE to draw contours
    for cnt in contours0:
        cv2.drawContours(frame, cnt, -1, (0,255,0), 3, 8)

    cv2.imshow('Frame',frame)
    if cv2.waitKey(30) == ord('q'):
    #lower the number higher the speed at which the frames are displayed
        break
        

cap.release()
cv2.destroyAllWindows()

error: C:\projects\opencv-python\opencv\modules\core\src\matrix.cpp:991: error: (-215) dims <= 2 && step[0] > 0 in function cv::Mat::locateROI


## Define shopping items(emperical threshold value)

In [4]:
cap = cv2.VideoCapture("C:\\Users\\212458792\\Music\\Downloads\\BasketVideos\\v1.mp4")
fgbg = cv2.createBackgroundSubtractorMOG2(detectShadows = True) #Create the background substractor
kernelOp = np.ones((3,3),np.uint8)
kernelCl = np.ones((11,11),np.uint8)
areaTH = 500

while(cap.isOpened()):
    ret, frame = cap.read() #read a frame
    
    fgmask = fgbg.apply(frame) #Use the substractor
    try:
        ret,imBin= cv2.threshold(fgmask,200,255,cv2.THRESH_BINARY)
        #Opening (erode->dilate) para quitar ruido.
        mask = cv2.morphologyEx(imBin, cv2.MORPH_OPEN, kernelOp)
        #Closing (dilate -> erode) para juntar regiones blancas.
        mask =  cv2.morphologyEx(mask , cv2.MORPH_CLOSE, kernelCl)
    except:
        #if there are no more frames to show...
        print('EOF')
        break
    _, contours0, hierarchy = cv2.findContours(mask,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_NONE)
    for cnt in contours0:
        cv2.drawContours(frame, cnt, -1, (0,255,0), 3, 8)
        area = cv2.contourArea(cnt)
        print(area)
        if area > areaTH:
            #################
            #   TRACKING    #
            #################            
            M = cv2.moments(cnt)
            cx = int(M['m10']/M['m00'])
            cy = int(M['m01']/M['m00'])
            x,y,w,h = cv2.boundingRect(cnt)
            cv2.circle(frame,(cx,cy), 5, (0,0,255), -1)            
            img = cv2.rectangle(frame,(x,y),(x+w,y+h),(0,255,0),2)
        
    cv2.imshow('Frame',frame)
    
    #Abort and exit with 'Q' or ESC
    k = cv2.waitKey(30) & 0xff
    if k == 27:
        break

cap.release() #release video file
cv2.destroyAllWindows() #close all openCV windows

149095.0
149279.0
8.0
373.5
4.0
8.0
12.5
75.5
11.5
81.0
55.0
58.5
4.0
40.5
6.0
266.5
4.0
42.0
111.5
103.0
84.0
8.0
234.5
57.5
4.0
31.0
392.5
6626.5
26.5
6.0
280.0
15.5
160.0
6.0
29.0
59.5
9.0
126.5
91.5
6.0
32.0
6.0
6.0
6.0
31.5
41.0
4.0
11.5
181.0
4.0
29.0
542.5
14.0
224.5
4.0
4605.5
12.0
20.5
59.5
23.5
4.0
34.5
10.5
76.5
20.5
10.0
234.5
26.0
4.0
6.0
9.0
4.0
27.0
100.0
20.0
172.5
125.0
8.5
114.0
28.0
4.0
140.5
4.0
28.5
9.0
1400.5
8.0
51.0
70.0
33.0
15.0
29.0
4.0
663.5
478.0
60.5
21.0
12.5
21.5
51.5
6.0
21.5
40.5
6.0
6.0
21.0
22.5
15.0
30.0
149.0
69.5
67.0
9.0
217.5
15.0
6.0
1289.0
43.5
4.0
17.5
19.0
12.0
960.5
4.0
4.0
6.0
4.0
4.0
316.5
12.5
32.5
4.0
6.0
15.0
4.0
29.0
6.0
6.0
6.0
10.0
8.0
36.0
131.5
14.5
84.0
59.0
916.5
4.0
3305.0
4.0
4.0
25.5
374.5
36.5
996.0
76.0
9.0
6.0
4.0
381.5
19.5
4.0
13.5
32.0
16.0
10.0
19.5
15.5
11.5
27.5
13.0
8.5
56.0
6.0
18.5
4.0
19.5
79.5
4.0
4.0
50.0
1469.5
4867.5
4.0
12.5
12.5
71.5
14.0
312.0
8.5
64.0
410.0
60.0
36.5
63.5
178.5
18.0
21.0
4.0
20.5
43.5
22.

8.5
3957.5
10.5
10.5
4.0
10.5
13.5
18.5
12.0
20224.5
9.0
6.0
30.5
66.0
99.0
4.0
21.0
4.0
4.0
23.0
4.0
9.0
24.0
6.0
6.0
21.0
113.5
9.0
4.0
17.5
6.0
8.5
10.0
4.0
77.0
4.0
6.0
9.0
61.0
6.0
12.5
20.5
341.5
4.0
28.0
14.5
4.0
4.0
4.0
4.0
4.0
4.0
9.0
10.5
30.0
4.0
59.5
200.5
129.5
34.0
4.0
63.0
9.0
8.0
14.5
6.0
18.5
4.0
6.0
6.0
4.0
25.0
6.0
16.5
33.5
9.0
4059.0
290.5
14.0
53.0
4.0
196.5
8.0
26.5
72.5
1993.0
8.5
2329.5
377845.0
2.0
4.0
381.0
320.0
143.5
12.0
6.0
2832.5
6.0
4.0
6.0
4.0
4.0
4.0
12.0
39.0
26.0
12827.0
6.0
6.0
367.0
14.5
4.0
101.5
4.0
66.5
24.0
4.0
13.0
6.0
6.0
154.0
31.5
4.0
6.0
4.0
80.0
4.0
9.0
4.0
20.5
14.5
6.0
15655.0
6.0
8.0
16.0
12.0
4.0
4.0
152.0
4.0
35.5
4.0
23.5
10.5
4.0
8.0
4.0
14.5
12.5
6.0
6.0
4.0
4.0
12.5
16.5
21.0
12.5
4.0
9.0
18.5
16.0
36.5
1879.0
4.0
59.5
4.0
40.0
6.0
23.0
136.0
8.0
83.0
8.5
1725.5
2226.0
12.0
346032.0
8.5
76.0
1353.5
4.0
4.0
3374.5
125.5
4.0
12.5
6.0
16.0
22.5
25.5
4.0
1389.0
12.0
10625.5
38.0
4.0
6.0
22.0
77.0
9.0
6.0
16.0
35.5
24.0
45.0
19.0
13.

4.0
27.0
6.0
9.0
45.5
17.5
4.0
15.0
25.5
49.5
20.0
6.0
10.0
12.0
6.0
20.0
8.0
39.5
4.0
16.5
6.0
199.5
19.0
23.5
11.5
8.5
10.5
4.0
10.5
4.0
4.0
6.0
14.5
9.0
4.0
4.0
4.0
98.0
4.0
14.5
22.5
35.0
4.0
6.0
14.0
136.5
12.0
8.5
14.5
26.5
4.0
9.0
4.0
4.0
4.0
51.0
6.0
6.0
24.5
4.0
4.0
449.5
21.0
6.0
17.5
12.0
52.5
4.0
19.0
4.0
6.0
4.0
8.0
27.5
926.0
51.0
226.0
6.0
13.5
6.0
6.0
270.5
39.5
22.5
48.0
1223.5
6.0
39.5
1017.0
4.0
17.5
4.0
86220.0
4.0
15.0
11.0
14.5
6.0
22.0
41.0
256.0
8.5
545.5
4.0
8.5
23.0
12.0
4.0
18.5
6.0
18.5
4.0
10.5
42.5
10.0
36.0
12.0
18.0
15.0
6.0
373.5
6.0
82.5
6.0
2455.5
4.0
18.0
4.0
11.5
53.0
37.0
23.0
213.0
260.5
62.5
110.0
235.0
4.0
8.0
9.0
1613.5
2096.0
40807.0
47581.5
28.0
12.0
45.5
17.5
25.5
10.5
6.0
20.5
10.5
6.0
40.5
95.0
21.0
6.0
6.0
13.0
19.5
8.5
17.0
4.0
6.0
6.0
6.0
4.0
4.0
4.0
43.0
8.5
9.0
24.0
4.0
4.0
4.0
43.5
25.0
31.0
6.0
4.0
47.0
12.0
8.0
4.0
4.0
24.0
349.5
20.0
4.0
8.5
24.0
11.0
21.5
4.0
4.0
34.5
896.0
4.0
25.5
6.0
8.5
4.0
254.5
23.5
12.0
840.5
6.0
12.5
153.

6.0
1470.0
9.0
8.0
19.0
8.0
40.5
111.0
77.5
4.0
289.5
15.0
4.0
434.5
104.5
4.0
6.0
53595.0
9.0
4.0
1501.5
31.5
8.0
23.0
56.5
111.0
75.0
4.0
250.5
15.5
123.5
502.5
101.0
44.5
4.0
21.0
53601.0
933.5
4.0
8.0
43.5
13.0
4.0
6.0
6.0
30.5
48.5
6.0
498.0
8.0
141.5
4.0
6.0
12.0
51280.5
933.5
4.0
8.0
38.0
4.0
4.0
6.0
6.0
30.5
40.5
6.0
438.5
8.0
141.5
34.5
9.0
14.0
51101.0
757.5
8.0
36.0
15.0
4.0
4.0
6.0
458.5
187.0
12.5
4.0
46.5
33.5
49339.5
8.0
554.5
4.0
8.0
42.0
139.0
4.0
4.0
6.0
65.0
729.5
4.0
23.5
55.0
49150.0
524.5
4.0
17.0
4.0
8.0
66.0
371.5
4.0
6.0
4.0
6.0
4.0
4.0
1036.5
4.0
26.5
49307.0
479.0
6.0
12.0
19.0
4.0
8.0
62.0
393.0
4.0
4.0
4.0
6.0
4.0
4.0
4.0
1023.0
4.0
26.0
49270.0
4.0
269.5
4.0
11.0
4.0
8.0
51.0
369.5
4.0
4.0
8.5
4.0
957.5
13.0
26.0
38.5
49591.0
8.5
292.0
11.0
4.0
8.0
72.0
376.5
4.0
8.5
4.0
6.0
916.0
6.0
46.0
15235.0
34827.0
4.0
4.0
235.0
4.0
4.0
8.0
79.5
349.0
4.0
4.0
6.0
897.5
6.0
6.0
46.0
51670.0
122.5
14.0
4.0
8.0
27.5
54.0
274.0
11.0
4.0
4.0
801.5
6.0
40.5
52158.5
98.0
4

8.0
20.0
55.0
32.0
150.5
21.0
27.0
55.5
6.0
10.0
17.5
1711.0
6.0
4.0
275.5
20.5
42.0
16.5
31.0
4.0
50.5
9.0
15.0
18.0
6.0
6.0
4.0
16.0
13.0
4.0
9.0
15.0
6.0
13.0
28.5
13.0
381.5
4.0
16.0
8.5
4.0
4.0
122.5
4.0
4.0
27.5
242.0
27.0
4.0
9.0
4.0
4.0
12.0
52.5
10.0
12.0
50.5
4.0
6.0
12.0
6.0
88.5
4.0
10.0
12.0
4.0
6.0
9.0
265.5
6.0
4.0
40.5
10.5
30.5
184.0
6.0
9.0
592.0
6.0
4.0
417.0
112.0
9.0
6.0
4.0
65.0
14.5
8.0
16.5
11.5
3349.0
9.0
26.5
8.5
311.5
4.0
30.0
98.0
41.5
6.0
32.0
36.0
4.0
12.0
16.5
6.0
4.0
32.5
6.0
18.5
4.0
4.0
30.0
193.5
4.0
4.0
11492.0
4.0
6.0
8.0
5.0
379221.5
8.0
20.0
55.0
32.0
150.5
21.0
27.0
55.5
6.0
10.0
17.5
1714.0
6.0
4.0
271.5
20.5
42.0
16.5
4.0
50.5
9.0
11.0
18.0
6.0
6.0
4.0
16.0
13.0
4.0
9.0
15.0
6.0
13.0
28.5
13.0
272.5
4.0
16.0
8.5
4.0
4.0
122.5
4.0
4.0
27.5
18.5
242.0
4.0
9.0
10.0
4.0
12.0
52.5
19.5
10.0
8.0
25.0
4.0
6.0
12.0
6.0
88.5
10.0
29.0
8.5
24.0
4.0
20.0
9.0
258.0
6.0
4.0
40.5
10.5
30.5
174.0
6.0
9.0
592.0
6.0
4.0
417.0
9.0
6.0
4.0
65.0
4.0
8.0
16.5
4.0
2

...

19.5
10.0
37.0
16.0
17.0
13.0
22.5
4.0
4.0
11.5
94.0
130.0
581.5
6.0
70634.5
1731.5
29.0
6.0
3425.0
11333.0
8.5
20.0
6.0
10.0
68.5
4.0
23.5
20.0
168.0
4.0
6.0
29.0
50.0
8.5
13.0
389.5
43.0
705.5
6.0
43.0
1663.0
6.0
4.0
70514.0
36.5
2.0
5967.5
2847.0
8.5
4.0
22.5
6.0
10.0
68.0
4.0
23.5
14.5
168.0
4.0
4.0
6.0
29.0
90.0
12.5
16.0
391.5
27.5
705.5
6.0
43.0
1686.5
4.0
4.0
70383.0
42.5
10.0
5910.0
2848.0
18.0
6.0
22.5
6.0
28.0
119.5
6.0
35.0
43.5
88.5
22.0
361.0
4.0
6.0
85.0
8.0
20.0
4.0
20.0
4.0
506.5
288.5
4.0
4.0
104.5
4.0
50.0
26.5
29.5
19.5
71119.0
4.0
46.5
8640.5
8.5
11.0
38.0
4.0
20.0
139.5
4.0
15.0
80.5
30.0
4.0
889.5
93.0
19.5
8.0
19.5
4.0
507.5
128.5
4.0
1024.0
228.5
71144.5
20.5
9.0
14010.0
15.5
11.0
34.5
4.0
27.0
137.5
4.0
15.0
86.5
36.5
4.0
4.0
4.0
1036.5
92.5
19.5
8.0
4.0
23.0
4.0
512.5
140.0
4.0
239.0
12.5
10.5
71215.5
19.5
28.0
15005.5
15.0
16.5
37.0
13.5
4.0
29.0
4.0
123.5
8.0
15.0
72.0
4.0
6.0
4.0
6.0
992.5
6.0
107.0
8.5
4.0
20.5
443.5
114.5
4.0
24.0
10.5
71016.0
4.0
4.0
80

14.5
14.0
143.5
11.5
4.0
19.0
34.5
777.0
4.0
70.5
46.0
4.0
4.0
4.0
6.0
14.5
48.0
267.5
12.0
6.0
14.0
244.0
129.5
6.0
79.0
9.0
12.0
6.0
11.0
8.0
50117.0
12.0
15.0
13152.5
13.5
4.0
140.0
6.0
4.0
13.0
30.5
29.5
4.0
603.0
78.0
83.5
56.0
14.5
48.0
4.0
162.5
12.0
6.0
6.0
108.5
181.5
63.0
14.5
6.0
51.5
9.0
12.0
6.0
6.0
11.0
6.0
48617.0
5784.0
14.5
14.0
2.0
41.5
7931.0
12.0
4.0
126.0
6.0
4.0
6.0
26.0
4.0
593.5
72.5
75.5
53.5
4.0
14.5
4.0
6.0
4.0
162.5
9.0
10.0
12.0
6.0
62.0
200.5
4.0
71.5
10.5
14.5
6.0
30.0
12.0
4.0
4.0
47711.0
6183.5
27.0
81.0
6.0
7288.0
14.5
4.0
4.0
141.0
6.0
4.0
4.0
18.0
61.0
4.0
442.5
55.0
104.0
52.5
4.0
8.5
4.0
6.0
4.0
160.5
9.0
4.0
12.0
12.0
6.0
4.0
200.0
4.0
103.5
71.5
4.0
12.5
28.0
4.0
22.5
4.0
4.0
46273.0
6070.5
18.5
92.0
72.5
83.0
4884.5
11.5
8.0
4.0
115.5
4.0
23.0
18.0
72.5
4.0
392.5
56.5
103.5
52.0
4.0
6.0
4.0
6.0
4.0
160.5
9.0
4.0
9.0
12.0
4.0
98.5
30.5
22.0
4.0
71.5
10.0
896.5
6.0
45437.5
169.5
5532.0
9.0
9.0
4.0
24.0
4.0
283.0
2903.5
6.0
8.0
4.0
103.5
11.0
18.0


47.5
4.0
36.0
23.5
4.0
71.5
9.0
20.0
44.0
2417.5
8.5
44.0
31.0
6.0
4.0
851.5
18.0
4.0
9.0
6.0
22.0
47.5
16.0
187.0
6.0
58.0
152.0
30.0
502.5
10.0
532.0
51.5
4.0
4.0
4.0
10.5
4.0
21.5
751.0
47.0
28.5
15.0
53.0
107.0
158.5
81.5
112.5
50.5
4.0
13.0
241262.5
74.0
6.0
8.5
4.0
28.0
6.0
55.5
4.0
6.0
30.5
9.0
2373.0
14.0
155.0
9.0
38.0
14.5
4.0
1018.0
15.0
9.0
18.0
6.0
162.5
9.0
4.0
9.0
10.0
12.0
46.5
4.0
209.0
14.0
83.5
6.0
21.5
6.0
9.0
6.0
22.5
39.5
379.0
36.5
15.0
6.0
24.5
108.0
68.5
257.0
4.0
17.5
251585.5
90.0
6.0
42.0
190.0
4.0
4.0
4.0
83.5
6.0
25.0
18.0
12.0
1988.0
4.0
207.0
4.0
111.0
4.0
4.0
670.5
38.0
4.0
4.0
18.0
170.5
9.0
15.0
45.0
6.0
248.5
4.0
502.0
4.0
10.0
14.0
536.5
6.0
65.0
16.5
4.0
4.0
8.5
10.5
6.0
38.0
563.5
19.0
6.0
204.0
9.0
93.5
84.5
14.0
4.0
36.5
8.5
4.0
14.0
253272.0
71.5
30.0
79.5
72.5
10.0
20.0
68.5
4.0
9.0
4.0
19.0
1804.5
4.0
8.5
182.0
6.0
6.0
15.5
55.0
812.5
4.0
10.0
258.0
4.0
110.5
6.0
22.5
170.5
8.0
216.5
241.5
458.0
44.0
744.5
97.5
89.5
53.0
10.0
13.0
6.0
29.0
34

13.0
4.0
14.0
9.0
9.0
8.5
6.0
8.5
10.0
32.0
121.5
59.5
144.5
32.0
4.0
25.5
25.0
92.5
91.5
6.0
44.0
10.0
171.0
6.0
4.0
4.0
1450.5
115.0
4.0
51.0
168.5
34.5
20.0
89.5
4.0
8.5
6.0
438.0
189.5
155.5
15.5
10.0
299.5
4.0
6.0
4.0
4.0
9.0
17.0
472.5
54.5
15.0
243.0
12.0
8.0
6.0
4.0
174720.5
8.5
63.5
4.0
6.0
6.0
16.0
4.0
6.0
54.5
4.0
6.0
15.0
129.0
12.0
63.5
10.0
8.0
6.0
10.0
6.0
4.0
4.0
646.5
18147.5
4.0
4.0
4.0
4.0
90.0
11.5
9.0
9.0
9.0
6.0
8.5
10.0
30.0
121.5
67.5
140.0
35.0
9.0
4.0
25.0
92.5
86.5
9.0
6.0
44.0
10.0
48.0
44.0
180.0
18.0
1962.0
115.5
4.0
72.5
164.5
175.5
39.0
25.0
82.0
143.5
24.0
8.5
30.0
6.0
361.0
191.5
108.5
12.0
8.0
298.5
4.0
4.0
10.5
4.0
4.0
493.5
54.5
15.0
190.5
4.0
13.0
8.0
6.0
10.5
74.0
4.0
4.0
4.0
37.5
6.0
14.5
54.5
4.0
187.0
175648.5
18.0
11.0
8.0
8.0
6.0
6.0
4.0
4.0
462.0
18937.5
8.0
9.0
4.0
4.0
40.0
9.0
6.0
9.0
6.0
8.5
10.0
30.0
124.0
98.0
70.0
32.0
4.0
17.5
56.5
4.0
77.5
4.0
22.0
9.0
148.0
78.5
160.0
4.0
18.0
24.5
2160.0
115.5
4.0
75.5
167.5
230.0
40.5
46.0
61.0
20

134.0
199.0
97.5
4.0
109.5
58.5
10.0
74.5
60.5
142.0
30.5
10.0
1833.0
8.5
56.0
35.0
6.0
9.0
24.5
42.0
66.0
58.0
4.0
13.0
95.0
4.0
13.5
6.0
19.0
22.0
4.0
12.0
23.0
34.5
12.5
6.0
16.0
4.0
24.5
13.0
4.0
30.0
4.0
4.0
17.0
8.5
235.0
6.0
104.5
34.0
4.0
4.0
8.0
10.0
78.5
22.0
6.0
11.0
4.0
34.5
4.0
66.5
12.5
10.0
4.0
56.0
4.0
4.0
6.0
4.0
6.0
13.0
21.5
319.0
489.5
8.0
13.0
190.5
21.5
4.0
4.0
150.5
4.0
4.0
4.0
9.0
13.5
4.0
4.0
4.0
10.0
61.0
10.0
49.0
119.5
8.0
4.0
6.0
9.0
4.0
4.0
4.0
41.0
4.0
148599.0
111.0
28.0
95.5
55.0
42.0
73.5
48.5
56.5
36.0
122.5
24.0
10.0
1864.5
8.5
56.0
35.0
6.0
9.0
22.5
19.0
66.0
56.0
4.0
11.0
93.0
4.0
13.5
6.0
25.0
22.0
4.0
4.0
23.0
34.5
19.5
6.0
16.0
24.5
13.0
4.0
30.0
4.0
4.0
4.0
17.0
235.0
6.0
104.5
34.0
4.0
4.0
6.0
10.0
78.5
4.0
22.0
4.0
11.0
27.5
4.0
52.0
105.5
12.5
10.0
4.0
56.0
4.0
4.0
6.0
11.0
21.5
235.5
462.5
4.0
8.0
13.0
16.5
126.5
34.5
8.5
4.0
4.0
148.0
4.0
8.0
9.0
13.5
4.0
4.0
10.0
44.0
10.0
47.0
119.5
4.0
6.0
9.0
4.0
4.0
41.0
4.0
146033.5
81.5
28.0
95.5
43